<a href="https://colab.research.google.com/github/ds4geo/ds4geo/blob/master/WS%202020%20Course%20Notes/Session%204.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# To Do

* time series intro/interpolation mini-lecture and notes



* References


* Create separate solutions notebook
* Enable output cells
* Final go-through and checks

# **Data Science for Geoscientists - Winter Semester 2020**
# **Session 4 - Time Series**
In the previous sessions and assignments, we've covered basic data handling, maniuplation and visualisation. In this session we will go deeper into working with time series, especially two frequently encountered topics: interpolation and filtering. We will use a monitoring dataset from Spanagel cave.

# Part 4.1 - Data Icebreaker - *Discussion*
Hypothesising about oceans.

# Part 4.2 - Cave monitoring data excercise part 1 - *Workshop*
*intro*
*Link to poster explanation plus give short intro - delta ray, intending to understand cave system/dynamics*


*come to  reason for interpolation*

## Part 4.2.1 - Load the cave monitoring data
Three datasets are available: Cave temperature, Outside air temperature, and Cave CO2 measurements (concentration, d13C, d18O).

The data sets are located as follows:
* Cave air temperature:
 * "https://github.com/ds4geo/ds4geo/raw/master/data/timeseries/Au%C3%9Fenluft%2BEingangslabyrinth.xlsx"
 * Sheet: "Daten3"
 * Time column: A, data column: G
* Outside air temperature:
 * "https://github.com/ds4geo/ds4geo/raw/master/data/timeseries/Au%C3%9Fenluft%2BEingangslabyrinth.xlsx"
 * Sheet: "Daten3"
 * Time column: I, data column: J
* Cave CO2 measurements:
 * https://github.com/ds4geo/ds4geo/raw/master/data/timeseries/CO2%20_compiled.xlsx"
 * Sheet: "Data Stream (2)
 * Time column: A, d13C column: C, d18O column: D, ppm CO2 column: E
                         sheet_name="Data Stream (2)", names=["time", "unknown", "d13C", "d18O", "ppm CO2", "ctime"]

Load all three datasets using Pandas

In [ ]:
# ANSWERS
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import numpy as np

In [ ]:
#ANSWERS
air_temp = pd.read_excel("https://github.com/ds4geo/ds4geo/raw/master/data/timeseries/Au%C3%9Fenluft%2BEingangslabyrinth.xlsx", sheet_name="Daten3", usecols=[0,6], names=["time", "air_temp"])
cave_temp = pd.read_excel("https://github.com/ds4geo/ds4geo/raw/master/data/timeseries/Au%C3%9Fenluft%2BEingangslabyrinth.xlsx", sheet_name="Daten3", usecols=[8,9], names=["time", "cave_temp"])
cave_co2 = pd.read_excel(r"https://github.com/ds4geo/ds4geo/raw/master/data/timeseries/CO2%20_compiled.xlsx",
                         sheet_name="Data Stream (2)", names=["time", "unknown", "d13C", "d18O", "ppm CO2", "ctime"])

##Part 4.2.2 - Plot the data to get an overview
Make a few plots to get an overview of the data.

In [ ]:
# ANSWERS
fig, ax = plt.subplots(figsize=(18,10))
ax2 = ax.twinx()
air_temp.plot("time", "air_temp", ax=ax)
cave_temp.plot("time", "cave_temp", ax=ax)
cave_co2.plot("time", "ppm CO2", ax=ax2, color="r")
ax.set_xlim([datetime.date(2015,9,17), datetime.date(2015,11,20)])

##Part 4.2.3 - Create a sub-set of the data for analysis
Create a sub-set of each dataset containing only the data between September and November 2015.

Hint: set the dataset indexes to the time column (e.g. df.index = df.time) and then you can index by date easily (df["2015-01-01" : "2015-02-01"]).

In [ ]:
# ANSWERS
# get data only in range
air_temp.index = air_temp.time # make time the index
air_select = air_temp["2015-9-17" : "2015-11-20"]

cave_temp.index = cave_temp.time
cave_select = cave_temp["2015-9-17" : "2015-11-20"]

cave_co2.index = cave_co2.time
co2_select = cave_co2["2015-9-17" : "2015-11-20"]

## Part 4.2.4 - Check the number of samples/sampling rate
To do further analysis, it helps if all three datasets have the same amount of data with corresponding timestamps and sampling rate. Check the lenght of each and the sampling rate of each.

Hint: you will need to using indexing, and the .value_counts() method will be useful.

In [ ]:
#ANSWERS
# Check lengths
print("air data shape:", air_select.shape)
print("cave data shape:", cave_select.shape)
print("co2 data shape:", co2_select.shape)
# Check sampling rate/timing
print("air sampling rate:",(air_select.index[1:] - air_select.index[:-1]).value_counts())
print("cave sampling rate:",(cave_select.index[1:] - cave_select.index[:-1]).value_counts())
print("co2 sampling rate:",(co2_select.index[1:] - co2_select.index[:-1]).value_counts())
# air is 1 hour, cave temp is 15 mins, co2 is all over the place. Mostly around 5 minutes, but sometimes 10, or much longer.
# If we resample to the cave temp 15 mins, then we are upsampling air temp, and downsampling the co2 data

# Part 4.3 - Timeseries and interpolation theory - *Mini-Lecture*
You will see from the workshop that the rate of data sampling is different in each of the datasets, and is even inconsistent within some of them. To analyse them further, we need them to be at the same rate, so we need to resample or interpolate them.

*add mini lecture notes*

*time series background*

*resampling vs interpolation*


# Part 4.4 - Introduction to SciPy - *Mini-lecture*
While Numpy handles multidimensional arrays and various mathematical operations on them, the SciPy library provides a wide range of more advanced functionality which is particularly useful for scientific data analysis.

It includes for example modules concerning linear algebra, regression/fitting, integration, signal processing, image manipulation and statistics.

SciPy can also refer to a collection of related libraries including Numpy, Pandas, Matplotlib and the SciPy library itself.

It contains a module called scipy.interpolate which we will use in the next section.



#Part 4.5 - Cave monitoring data excercise part 2 (resampling/interpolation) - *Workshop*
We will now resample the datasets so they have the same sampling rate, and that the data corresponds directly in terms of sampling time. We will resample everything to the same sampling rate and times as the cave temperature record.

In [ ]:
# For all of the below sections we will need the SciPy Interpolate module:
from scipy import interpolate

## Part 4.5.1 - Upsampling outside air temperature
The outside air temperature is sampled at a lower rate than the cave temperature, therefore we need to upsample the outside air temperature record.

Re-write the following pseudo-code to perform resampling by linear interpolation.

In [ ]:
'''
1. Create an interpolation object containing the outside air temperature data where:
We use the object "interp1d" (actually a class) from the Scipy interpolation 
<A> is the time column of the outside air temp data
   we cannot interpolate using times directly, so "to_numpy().astype(float) is a trick to convert the times to decimal numbers"
<B> is the temperature column outside air temp data
<C> is the type of interpolation - look at the in interp1d help documentation
<D> is the variable name to give to the interpolation object
'''
# <D> = interpolate.interp1d(<A>.to_numpy().astype(float), <B>, kind=<C>)

In [ ]:
'''
2. Perform the interpolation by providing the times from the cave temperature
data set. These are the times where we want to calculate surface air temperatures.
<E> is the time column of the cave temp record, converted to decimal numbers as
     in <A>
<F> is the resampled time-series
'''
#<F> = <D>(<E>)

In [ ]:
'''
3. Plot the resampled result and compare it to the original.
It will be helpful to visualise the datapoints themselves, so use data markers
in the plot.
'''

In [ ]:
#ANSWErS
interpf = interpolate.interp1d(air_select.time.to_numpy().astype(float), air_select.air_temp, kind="linear", fill_value="extrapolate")
air_interp = interpf(cave_select.time.to_numpy().astype(float))
fig, ax = plt.subplots(figsize=(15,10))
ax.plot(cave_select.index, air_interp, "x-")
ax.plot(air_select.index, air_select.air_temp, "o")
ax.set_xlim(datetime.date(2015,10,15), datetime.date(2015,10,16))
ax.set_ylim(-4, 3)


##Part 4.5.2 - Downsample the cave CO2 data
The cave CO2 data is at a much higher and variable sampling rate, so we will downsample it to the cave temperature sampling rate/timings.

## Part 4.5.3 - Test interpolation methods

In [ ]:
### ANSWERS
# Testing upsampling of air temps
kinds = ["linear", "nearest", "zero", "slinear", "quadratic", "cubic"]#, "previous", "next"]
intp_air = []
for k in kinds:
  interpf = interpolate.interp1d(air_select.time.to_numpy().astype(float), air_select.air_temp, kind=k, fill_value="extrapolate")
  air_interp = interpf(cave_select.time.to_numpy().astype(float))
  intp_air.append(air_interp)

fig, ax = plt.subplots(figsize=(15,10))
for i,l in zip(intp_air, kinds):
  ax.plot(cave_select.index, i, "x-", label=l)
ax.plot(air_select.index, air_select.air_temp, "o")
ax.set_xlim(datetime.date(2015,10,15), datetime.date(2015,10,16))
ax.set_ylim(-4, 3)
ax.legend()

#Part 4.6 - Introduction to data filtering - *Mini-lecture*
*also mention complex filters, but let them explore in workshop*

#Part 4.7 - Cave monitoring data excercise part 3 (filtering) - *Workshop*

# Part 4.8 - Advanced filtering - *Mini-lecture*
*well, more advanced - not going to FFT, etc. yet!

# Part 4.9 - Week 4 Assignment

# References

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import numpy as np
from scipy import interpolate

In [ ]:
air_temp = pd.read_excel("https://github.com/ds4geo/ds4geo/raw/master/data/timeseries/Au%C3%9Fenluft%2BEingangslabyrinth.xlsx", sheet_name="Daten3", usecols=[0,6], names=["time", "air_temp"])

In [ ]:
air_temp

In [ ]:
cave_temp = pd.read_excel("https://github.com/ds4geo/ds4geo/raw/master/data/timeseries/Au%C3%9Fenluft%2BEingangslabyrinth.xlsx", sheet_name="Daten3", usecols=[8,9], names=["time", "cave_temp"])

In [ ]:
cave_temp

In [ ]:
cave_temp.plot("time", "cave_temp")
air_temp.plot("time", "air_temp")

In [ ]:
cave_co2 = pd.read_excel(r"https://github.com/ds4geo/ds4geo/raw/master/data/timeseries/CO2%20_compiled.xlsx",
                         sheet_name="Data Stream (2)", names=["time", "unknown", "d13C", "d18O", "ppm CO2", "ctime"])

In [ ]:
cave_co2

In [ ]:
fig, ax = plt.subplots(figsize=(18,10))
ax2 = ax.twinx()
air_temp.plot("time", "air_temp", ax=ax)
cave_temp.plot("time", "cave_temp", ax=ax)
cave_co2.plot("time", "ppm CO2", ax=ax2, color="r")
ax.set_xlim([datetime.date(2015,9,17), datetime.date(2015,11,20)])

In [ ]:
# get data only in range
air_temp.index = air_temp.time # make time the index
air_select = air_temp["2015-9-17" : "2015-11-20"]

cave_temp.index = cave_temp.time
cave_select = cave_temp["2015-9-17" : "2015-11-20"]

cave_co2.index = cave_co2.time
co2_select = cave_co2["2015-9-17" : "2015-11-20"]

In [ ]:
# Check lengths
print("air data shape:", air_select.shape)
print("cave data shape:", cave_select.shape)
print("co2 data shape:", co2_select.shape)
# All different

In [ ]:
# Check sampling rate/timing
print("air sampling rate:",(air_select.index[1:] - air_select.index[:-1]).value_counts())
print("cave sampling rate:",(cave_select.index[1:] - cave_select.index[:-1]).value_counts())
print("co2 sampling rate:",(co2_select.index[1:] - co2_select.index[:-1]).value_counts())
# air is 1 hour, cave temp is 15 mins, co2 is all over the place. Mostly around 5 minutes, but sometimes 10, or much longer.
# If we resample to the cave temp 15 mins, then we are upsampling air temp, and downsampling the co2 data

In [ ]:
# Testing upsampling of air temps
kinds = ["linear", "nearest", "zero", "slinear", "quadratic", "cubic"]#, "previous", "next"]
intp_air = []
for k in kinds:
  interpf = interpolate.interp1d(air_select.time.to_numpy().astype(float), air_select.air_temp, kind=k, fill_value="extrapolate")
  air_interp = interpf(cave_select.time.to_numpy().astype(float))
  intp_air.append(air_interp)

In [ ]:
air_select.time.to_numpy()

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
for i,l in zip(intp_air, kinds):
  ax.plot(cave_select.index, i, "x-", label=l)
ax.plot(air_select.index, air_select.air_temp, "o")
ax.set_xlim(datetime.date(2015,10,15), datetime.date(2015,10,16))
ax.set_ylim(-4, 3)
ax.legend()

In [ ]:
# Downsampling of co2 data
kinds = ["linear", "nearest", "zero", "slinear", "quadratic", "cubic"]#, "previous", "next"]
intp_co2 = []
for k in kinds:
  interpf = interpolate.interp1d(co2_select.time.to_numpy().astype(float), co2_select["ppm CO2"], kind=k, fill_value="extrapolate")
  air_interp = interpf(cave_select.time.to_numpy().astype(float))
  intp_co2.append(air_interp)

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
ax.plot(co2_select.index, co2_select["ppm CO2"], "o")
for i,l in zip(intp_co2, kinds):
  ax.plot(cave_select.index, i, "x-", label=l)
#ax.set_xlim(datetime.date(2015,10,17), datetime.date(2015,10,18))
ax.set_xlim(datetime.datetime(2015,10,17, 9), datetime.datetime(2015,10,17,15))
ax.set_ylim(425, 465)
ax.legend()
# alternative downsampling to be addressed while filtering in next section - e.g. mean of window, decimate, etc.

In [ ]:
# Now interpolated, can do direct comparisons:
fig, ax = plt.subplots(figsize=(15,10))
ax.scatter(intp_co2[0], intp_air[0]-cave_select.cave_temp, marker="x")
ax.set_xlim([375, 450])

In [ ]:
cave_select.cave_temp.shape

In [ ]:
# Smoothing - need relevant data set to smooth!
#scipy-cookbook.readthedocs.io/items/SignalSmooth.html

In [ ]:
xdat = pd.read_csv("https://doi.pangaea.de/10.1594/PANGAEA.859980?format=textfile", header=48, sep="\t")


In [ ]:
xdat

In [ ]:
fig, ax1 = plt.subplots(figsize=(12,8))
ax2 = ax1.twinx()
ax3 = ax1.twinx()
ax4 = ax1.twinx()
xdat.plot("Depth [m]", "Al [cps]", ax=ax1, color="C0")
xdat.plot("Depth [m]", "K [cps]", ax=ax2, color="C1")
xdat.plot("Depth [m]", "Ti [cps]", ax=ax3, color="C2")
xdat.plot("Depth [m]", "V [cps]", ax=ax4, color="C3")
ax1.set_xlim(0.4,0.5)

In [ ]:
xdat.columns

In [ ]:
filter_length = 5
filter = np.ones(filter_length ) / filter_length

In [ ]:
filter

In [ ]:
Vf = np.convolve(xdat["V [cps]"],filter)

In [ ]:
Vf

In [ ]:
%matplotlib notebook
fig, ax = plt.subplots(figsize=(12,8))
ax.plot(xdat["Depth [m]"], Vf[2:-2])
ax.plot(xdat["Depth [m]"], xdat["V [cps]"])
ax.set_xlim(0.4,0.5)
plt.show()

In [ ]:
'''
Rule of thumb:
Matplotlib for static graphics, quick plotting and publication quality graphics
Plotly for interactive or static/publication quality
Bokeh for interactive/web plots
Seaborn for quick but powerful stats plots
Altair - different approach to making plots based on how we reason about data
https://colab.research.google.com/notebooks/charts.ipynb#scrollTo=N-u5cYwpS-y0

'''
# Lets try bokeh
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

# Call once to configure Bokeh to display plots inline in the notebook.
output_notebook()

In [ ]:
p = figure()
p.line(xdat["Depth [m]"], xdat["V [cps]"], line_color="black")
for fl, col in zip([3,5,9,15],["green", "indigo", "blue", "coral"]):
  filter = np.ones(fl ) / fl
  Vf = np.convolve(xdat["V [cps]"],filter)
  p.line(xdat["Depth [m]"], Vf[int((fl-1)/2):-int((fl-1)/2)], line_color=col)

show(p)

In [ ]:
# But what about not simple flat moving averages... try different filter shapes from cookbook
# Then also taking individual data points rather than proper smoothing.
# Also non-overlapping windows?

**Section X - Advanced Filters**


In [ ]:
# Compare results of these windows and also visualise the windows themselves: https://numpy.org/doc/stable/reference/routines.window.html